# Econometric UNFCCC Green Cities Commitment Analysis: USA
## Econometric Analysis

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor

In [2]:
df_all = pd.read_csv('allcities.csv')
df_unfccc = pd.read_csv('UNFCCC.csv')

In [3]:
def BooltoZeroOne(df):
    for column in df.columns:
        if df[column].dtype == bool:
            df[column] = df[column].astype(int)

In [4]:
BooltoZeroOne(df_all)
BooltoZeroOne(df_unfccc)

In [5]:
def FitLogistic(df, y_var, X_vars):
    df = df.dropna()
    X = df[X_vars]
    X = sm.add_constant(X)
    y = df[y_var]
    model = sm.Logit(y, X).fit(cov_type = 'HC3')
    print(model.summary())

def FitRandomForest(df, y_var, X_vars):
    df = df.dropna()
    X = df[X_vars]
    y = df[y_var]
    rf = RandomForestRegressor()
    model = rf.fit(X, y)
    return pd.DataFrame({
        'Feature':X_vars,
        'Importance':model.feature_importances_
    }).sort_values(by = 'Importance', ascending = False)

In [6]:
FitLogistic(df_all, y_var = 'UNFCCC', X_vars = ['population','redCounty','redState','unemploymentRate','povertyProp','tempDiff','numDisasters','avgEmissionsPerCapita','lessThanHighSchoolProp','medianHouseholdIncome'])

Optimization terminated successfully.
         Current function value: 0.051165
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                 UNFCCC   No. Observations:                14546
Model:                          Logit   Df Residuals:                    14535
Method:                           MLE   Df Model:                           10
Date:                Sat, 06 Apr 2024   Pseudo R-squ.:                  0.2865
Time:                        16:09:13   Log-Likelihood:                -744.24
converged:                       True   LL-Null:                       -1043.1
Covariance Type:                  HC3   LLR p-value:                5.273e-122
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -2.2981      1.065     -2.157      0.031      -4.386     

In [7]:
FitRandomForest(df_all, y_var = 'UNFCCC', X_vars = ['population','redCounty','redState','unemploymentRate','povertyProp','tempDiff','numDisasters','avgEmissionsPerCapita','lessThanHighSchoolProp','medianHouseholdIncome'])

,Feature,Importance
0,population,0.552670
8,lessThanHighSchoolProp,0.110932
9,medianHouseholdIncome,0.090781
4,povertyProp,0.067949
3,unemploymentRate,0.059093
7,avgEmissionsPerCapita,0.044048
5,tempDiff,0.033075
6,numDisasters,0.025380
1,redCounty,0.011991
2,redState,0.004081
